## **Q3 Code**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0  # Normalising pixel values to bring between 0 and 1

train_images_flat = train_images.reshape((train_images.shape[0], -1))
test_images_flat = test_images.reshape((test_images.shape[0], -1))

train_images_flat, val_images_flat, train_labels, val_labels = train_test_split(train_images_flat, train_labels, test_size=0.2, random_state=42)

model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images_flat, train_labels, epochs=10, batch_size=64, validation_data=(val_images_flat, val_labels))

test_loss, test_acc = model.evaluate(test_images_flat, test_labels)
print(f'Test accuracy: {test_acc}')


Epoch 1/10
750/750 [==============================] - 9s 11ms/step - loss: 0.2595 - accuracy: 0.9250 - val_loss: 0.1399 - val_accuracy: 0.9570
Epoch 2/10
750/750 [==============================] - 6s 8ms/step - loss: 0.0986 - accuracy: 0.9691 - val_loss: 0.1162 - val_accuracy: 0.9646
Epoch 3/10
750/750 [==============================] - 6s 8ms/step - loss: 0.0646 - accuracy: 0.9799 - val_loss: 0.0823 - val_accuracy: 0.9757
Epoch 4/10
750/750 [==============================] - 5s 7ms/step - loss: 0.0471 - accuracy: 0.9853 - val_loss: 0.0891 - val_accuracy: 0.9734
Epoch 5/10
750/750 [==============================] - 6s 8ms/step - loss: 0.0369 - accuracy: 0.9881 - val_loss: 0.0901 - val_accuracy: 0.9753
Epoch 6/10
750/750 [==============================] - 6s 9ms/step - loss: 0.0270 - accuracy: 0.9914 - val_loss: 0.1032 - val_accuracy: 0.9729
Epoch 7/10
750/750 [==============================] - 6s 8ms/step - loss: 0.0251 - accuracy: 0.9917 - val_loss: 0.0799 - val_accuracy: 0.9779
Epoch

## **Q4 Code**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

preprocess_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit models' input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

svhn_train_data = datasets.SVHN(root='./data', split='train', download=True, transform=preprocess_transform)
svhn_test_data = datasets.SVHN(root='./data', split='test', download=True, transform=preprocess_transform)

subset_indices = torch.randperm(len(svhn_train_data))[:len(svhn_train_data)//4] # Used only 1/4th data to reduce computation time
svhn_train_subset = torch.utils.data.Subset(svhn_train_data, subset_indices)

train_loader = DataLoader(svhn_train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(svhn_test_data, batch_size=64, shuffle=False)

models_dict = {

    'AlexNet': models.alexnet(),
    'VGG-11': models.vgg11(),
    'ResNet-101': models.resnet101(),
    'ResNet-18': models.resnet18()
}

for model_name, model in models_dict.items():

    if 'resnet' in model_name.lower():
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)
    else:
        num_ftrs = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_ftrs, 10)

    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0003)

    num_epochs = 3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"{model_name} - Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"{model_name} - Accuracy on Test set: {(correct/total)*100:.2f}%")


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
AlexNet - Epoch 1, Loss: 2.154567156934572
AlexNet - Epoch 2, Loss: 0.9937725551037008
AlexNet - Epoch 3, Loss: 0.5388172589216498
AlexNet - Accuracy on Test set: 86.30%
VGG-11 - Epoch 1, Loss: 1.8837075731895527
VGG-11 - Epoch 2, Loss: 0.5992555627631809
VGG-11 - Epoch 3, Loss: 0.385102289323192
VGG-11 - Accuracy on Test set: 86.93%
ResNet-101 - Epoch 1, Loss: 2.3017203857674415
ResNet-101 - Epoch 2, Loss: 1.8214034018616228
ResNet-101 - Epoch 3, Loss: 0.6068229930222242
ResNet-101 - Accuracy on Test set: 84.98%
ResNet-18 - Epoch 1, Loss: 1.3367541245467156
ResNet-18 - Epoch 2, Loss: 0.4146528744012221
ResNet-18 - Epoch 3, Loss: 0.2930637184231954
ResNet-18 - Accuracy on Test set: 87.43%
